## Постановка задачи

Подчеркнуть из комментариев:
* Заинтересованность в определённых продуктах
* Сезонность (когда гарантия, участие в конкретном тендере)
* Лизинг (машины, оборудование, сезонность)

Итоговый результат - Краткая справка по клиенту.

`Контакт с ГД. Сказал есть 3 расчетных счета, больше не надо. Есть овер в Альфе, рефить не хотят. Депозиты не интересуют, так как % в ВТБ выгодней. Сам является Клиентом Совкомбанка, как физик, перезвонить через месяц`

Использование Named Entity Recognition для выделения сущностей таких как 'расчётные счета', 'овердрафт', 'депозиты' и т.д.

```python
bank_products = {
    'расчётные счета': [],
    'овердрафт': False,
    'депозиты': False
}

for ent in doc.ents:
    if 'расчётный счёт' in ent.text:
        bank_products['расчётные счета'].append(ent.text)
    elif 'овердрафт' in ent.text:
        bank_products['овердрафт'].append(ent.text)
```

```python
overdraft_info = {
    'банк': None,
    'сумма': None,
}

for ent on doc.ents:
    if 'овердрафт' in ent.text:
        overdraft_info['банк'] = ent.text
    elif 'млн руб' in ent.text:
        overdraft_info['сумма'] = ent.text
```

План проекта
* Обучаем на выделение сущностей PRODUCT
* Находим сущности в комментарии и по шаблону-паттернам находим, где он находится, сумму, отношение

TAG | MORPH | description
--- | --- | ---
ROOT | СКАЗУЕМОЕ |
NMOD | NOUN MODIFIER | зависимое от главного существительное слово ((в) альфе овер)
NSUBJ | NOMINAL SUBJECT | подлежащее (овердрафт устраивает, депозиты интересуют)
XCOMP | OPEN CLAUSAL COMPLEMENT| глагол / прилагательное, которое требует завершения фразы
OBL | OBLIQUE NOMINAL | обстоятельство ((в) лизинг интересует)
ADVMOD | ADVERBIAL MODIFIER | модификатор наречия, слово или фраза, которое уточняет значение наречия в предложении
CONJ | CONJUNCTION | союз
OBJ | OBJECT | существительно, на которое направлено действие
ADJ | ADJECTIVE | прилагательное

* Разметка по позитивным действиям и негативным действиям
* **Если сущность в предложении, то выделяем часть предложения до знака препинания**



In [ ]:
!pip install spacy
!pip install optuna
!python -m spacy download ru_core_news_md #
# pip install  ru_core_news_sm-3.7.0-py3-none-any.whl

import numpy as np
import pandas as pd
import optuna
import spacy
import time
from tqdm import tqdm
from spacy import displacy
from spacy.training import Example
from sklearn.utils import shuffle

nlp = spacy.load('ru_core_news_md')

ner = nlp.get_pipe('ner')
ner.add_label('PROD')

available_entities = nlp.get_pipe('ner').labels
print('Доступные сущности:', available_entities)

In [ ]:
def dict_to_form(dictlist):
    formed_list = []
    for elem in dictlist:
        for key, value in elem.items():
            doc = nlp(value)
            elem[key] = extract_main_idea(doc)
            formed_list.append(elem)
    return formed_list

#ФУНКЦИЯ ИЗВЛЕЧЕНИЯ ГЛАВНОЙ МЫСЛИ
def extract_main_idea(sentence):
    main_idea = []
    for token in sentence:
        if token.dep_ not in ['punct']:
            if token.dep_ == 'ROOT' or token.dep_ == 'conj':
                main_idea.append(token.text)
            elif token.dep_ == 'nsubj' and token.head.dep_ == 'ROOT':
                main_idea.append(token.text)
    return ' '.join(main_idea)

#СПИСОК СЛОВАРЕЙ -> СПИСОК СЛОВАРЕЙ С ОБРАБОТАННЫМИ КЛЮЧАМИ
data_to_train['main_idea'] = data_to_train['entlist'].apply(dict_to_form)

# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)

data_to_train[['text', 'label', 'main_idea']]

#ПЕРЕПИСЫВАЕМ ФУНКЦИЮ ИЗВЛЕЧЕНИЯ ГЛАВНОЙ МЫСЛИ
def extract_main_idea(sentence):
    main_idea = []
    for token in sentence:
        if not token.ent_type_:
            if token.dep_ == 'ROOT':
                main_idea.append(token.text)
            if token.head.dep_ == 'ROOT' and token.dep_ in ['XCOMP', 'NMOD', 'ADVMOD']:
                main_idea.append(token.text)
    return ' '.join(main_idea)

train_data = [
    ('Мы редко ходим в кино', {'heads': [2, 2, 2, 0, 2], 'deps': ['nsubj', 'advmod', 'ROOT', 'prep', 'advmod']}),
    ('Депозиты не интересуют', {'heads': [3, 3, 0], 'deps': ['nsubj', 'neg', 'ROOT']})

]

`.rfind` - это метод строки в Python, который выполняет обратный поиск подстроки в строке. Он возвращает **индекс** последнего вхождения подстроки. </br>
Если подстрока не найдена, возвращается -1 . </br>
В приведённых примерах метод `.rfind` используется для определения позиции знаков препинания, предшествующих упоминаниям сущностей.

```python
sentence_start = max(text.rfind('.', 0, token.idx), text.find(',', 0, token_idx), 0)
```
Использование `.rfind` позволяет определить позицию последнего вхождения знака препинания перед упоминанием сущности в тексте.

`.strip` - это метод в Python, который используется для удаления начальных (слева) и конечных (справа) пробелов или других указанных символов из строки.

In [ ]:
for index, row in comment_data.iterrows():
    text = row['comment']
    doc = nlp(text)
    for token in doc:
        for entity in ['овердрафт', 'бкл', 'гарантия', 'реф', 'депозит', 'рефить', 'лизинг']:
            if token.lemma_.lower() == entity:
                sentence_start = max(text.rfind('.', 0, token.idx), text.rfind(',', 0, token.idx), 0)
                min_nums = [text.find('.', token.idx, len(text)), text.find(',', token.idx, len(text)), len(text)]
                sentence_end = min(num for num in min_nums if num != -1)
                entity_text = text[sentence_start + 1 : sentence_end].strip()
                print(f'{entity}: {entity_text}')

#ПЕРЕХОДИМ К ИНСТРУМЕНТАМ SPACY, ТАК КАК СТАНДАРТНЫЕ ИНСТРУМЕНТЫ PYTHON ОБРЕЗАЮТ ТОКЕНЫ ВРОДЕ 9,8%

овердрафт: имеется овердрафт в ВТБ 40 млн руб под 9
реф: реф и вкл не интересует
овердрафт: полностью устраивает овердрафт
лизинг: кроме лизинга
лизинг: В лизинг интересует оборудование
рефить: рефить не хотят
депозит: Депозиты не интересуют


* [Пример похожей задачи на Kaggle](https://www.kaggle.com/code/rohitsingh9990/ner-training-using-spacy-ensemble/notebook)
* [Обсуждение решения задачи](https://www.kaggle.com/code/rohitsingh9990/ner-training-using-spacy-ensemble/comments)
* [Тренировочный датасет](https://www.kaggle.com/competitions/tweet-sentiment-extraction/data?select=train.csv)

Выделяем главную мысль из текста
```python
def extract_main_idea(sentence):
    main_idea = []
    for token in sentence:
        if token.dep_ not in ['punct', 'neg']:
            if token.dep_ == 'ROOT' or token.dep_ == 'conj':
                main_idea.append(token.text)
            elif token.dep_ == 'nsubj' and token.head.dep_ == 'ROOT':
                main_idea.append(token.text)
    return ' '.join(main_idea)

```
* aspect-based sentiment analysis
* spacy dependency parser: NER достанет аспект и все слова, которые к нему относятся
* spacy dependency parcer дообучать не нужно (это уже обученная модель и самому дообучить её не так просто), ей нужно пользоваться, чтобы извлекать морфологические зависимости в данных


Дообучение векторных представлений сводится к простой передаче текстовых данных для обновления векторных представлений. </br>
Обучение векторного пространства в spaCy не затрагивает independence parser и качество определения части речи напрямую. Векторное представление обучается с использованием нейронной сети, которая учится предсказать контекстное окружение слова в тексте. Это позволяет модели понимать семантические отношения между словами и использовать их для различные задачЮ такие как сравнение синонимов или классификация текста. </br>
Independence parser и определение части речи (Part-of-Speech tagging) в spaCy основаны на других компонентах модели, таких как морфологический анализатор и синтаксический анализатор. Эти компоненты модели обучаются отдельно и не зависят от обучения векторного пространства. </br>

```python
from spacy.util import compounding
from spacy.util import minibatch
```

```python
# Вычисляем косинусное расстояние между векторами
similarity = vector_cat.dot(vector_dog) / (vector_cat.norm() * vector_dog.norm())
```
```python
# Токенизируем и лемматизируем текст
processed_corpus = []
for doc in nlp.pipe(corpus):
    processed_corpus.append([token.lemma_ for token in doc])
```
Ниже пример функции, которая очищает текст от стоп-слов, пунктуации, лишних пробелов, чисел и возвращает лемматизированный текст.
```python
def lemmatize(doc):
    words = []
    for token in doc:
        if (token.is_stop != True) and (token.is_punct != True) and\
            (token.is_space != True) and (token.is_digit != True):
            words.append(token.lemma_)
    return ' '.join(words)
```
```python
def train_model(nlp, data, n_iter=10):
    for i in tange(n_iter):
        random.shuffle(data)
        losses = {}

        for batch in minibatch(data, size=compounding(4.0, 32.0, 1.001)):
            texts, - = zip(*batch)
            nlp.update(texts, losses = losses)

        print('Эпоха {}: Потери: {}'.format(i+1, losses))
```

```python
from spacy.training import Example

#ЗАМЕНИТЬ TRAIN_DATA НА РАЗМЕЧЕННЫЙ ДАТАСЕТ
for i in range(10):
    random.shuffle(train_data)
    losses = {}
    for text, annotations in train_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.5, losses=losses)
    print('Эпоха', i, ':', losses)
```



In [ ]:
data_to_train = pd.read_excel('train_data.xlsx')

# ФУНКЦИЯ ПРИВЕДЕНИЯ ДАННЫХ К ТИПУ ТЕКСТ - СУЩНОСТЬ
def get_train_data(data_to_train):
    train_data = []
    for index, row in data_to_train.iterrows():
        text = row.text
        selected_text = row.selected_text
        entity_list = []
        for elem in selected_text.split(sep = ','):
            elem = elem.strip()
            start = text.find(elem)
            end = start + len(elem)
            entity_list.append((start, end, 'PROD'))
        train_data.append((text, {'entities': entity_list}))
    return train_data

# ПОЛУЧЕНИЕ ДАННЫХ
train_data = get_train_data(data_to_train)

# ОБУЧЕНИЕ
for i in range(10):
    train_data = shuffle(train_data)
    losses = {}
    try:
        for text, annotations in train_data:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], drop=0.1, losses=losses)
    except ValueError as e:
        print(f'Ошибка при обновлении модели: {e}')
    print('Эпоха', i, ':', losses)

In [ ]:
training_data = [
    ('бк с гд, попросил набрать в пн', {"entities": []})
    ]

train_examples = []
for text, annotations in training_data:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotations)
    train_examples.append(example)

for epoch in range(10):
    train_examples = shuffle(train_examples)
    losses = {}
    for example in train_examples:
        nlp.update([example], drop = 0.1, losses = losses)
    print('Эпоха', epoch, ':', losses)

Эпоха 0 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 2.71537340282563e-12}
Эпоха 1 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 1.118470388899019e-12}
Эпоха 2 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 2.620586592219322e-09}
Эпоха 3 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 1.106385799660583e-10}
Эпоха 4 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 1.0771147732686626e-10}
Эпоха 5 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 1.9575938102622994e-14}
Эпоха 6 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 4.959736954821176e-12}
Эпоха 7 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 1.0012769523486468e-14}
Эпоха 8 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 2.2191134165643226e-12}
Эпоха 9 : {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 9.538061576222029e-14}


In [ ]:
doc = nlp('бк с гд. потребности в кредите.! лизинг нет потребности.')
[ent.text for ent in doc.ents]

['кредите', 'лизинг']

In [ ]:
entlist_to_form('бк с гд. потребности в кредите.! лизинг нет потребности.')

[{'кредит': 'потребности в кредите '}, {'лизинг': '! лизинг нет потребности '}]

In [ ]:
# ОСТАВЛЯЕМ ТОЛЬКО СУЩНОСТИ И ИХ ОКРУЖЕНИЕ
def entlist_to_form(comment):
    comment = comment.lower()
    partlist = []
    entlist = []
    # Пустая строки
    current_part = ''
    doc = nlp(comment)
    for token in doc:
        if (token.text == ',') or (token.text == '.'):
            partlist.append(current_part)
            current_part = ''
        else:
            current_part += token.text
            current_part += ' '
    # Если строка НЕ пустая, добавим часть предложения, в список
    if current_part:
        partlist.append(current_part)
    # Пройдём по ранее заготовленному списку partlist
    for part in partlist:
        for ent in doc.ents:
            if ent.label_ == 'PROD' and ent.text in part:
                entlist.append({ent.lemma_: part}) #ПРИВОДИМ К ФОРМАТУ СУЩНОСТЬ-ЗНАЧЕНИЕ
    return entlist

In [ ]:
data_to_train['label'] = data_to_train['text'].apply(lambda x: [(ent.lemma_, ent.label_) for ent in nlp(x).ents])
data_to_train['entlist'] = data_to_train['text'].apply(entlist_to_form)

data_to_train[['text', 'label', 'entlist']]

,text,label,entlist
0,"кт с бухгалтером, предложение лизинга и кик, ...","[(лизинг, PROD), (кик, PROD)]","[{'лизинг': 'предложение лизинга и кик '}, {'к..."
1,закупка неакутальна,"[(закупка, PROD)]",[{'закупка': 'закупка неакутальна '}]
2,"бк с фд, // есть потребность в лизинге и вкл 5...","[(лизинг, PROD), (вкл, PROD), (депозит, PROD),...",[{'лизинг': '// есть потребность в лизинге и в...
3,бк с гд. потребности в кредите.! лизинг нет по...,"[(кредит, PROD), (лизинг, PROD)]","[{'кредит': 'потребности в кредите '}, {'лизин..."
4,"кт, все также кредитование, депозиты, бг неинт...","[(кредитование, PROD), (депозит, PROD), (бг, P...","[{'кредитование': 'все также кредитование '}, ..."
...,...,...,...
191,"потребность в бг, вкс есть основная анкета, вс...","[(бг, PROD), (вкс, PROD)]","[{'бг': 'потребность в бг '}, {'вкс': 'вкс ест..."
192,"бк с гд, очень сильно связана с втб, не уйти о...","[(лизинг, PROD), (факторинг, PROD)]",[{'лизинг': 'пробовать лизинг + факторинг пред...
193,"общение с евгением, сказал звонить после новог...","[(деньга, PROD), (кик, PROD)]","[{'деньга': 'деньги нужны на пол года '}, {'ки..."
194,ответил фд. в ближайшей перспективе не будут м...,"[(гарантия, PROD), (кредит, PROD)]",[{'гарантия': 'гб принимает решение по банковс...


Ниже пример функции, которая очищает текст от стоп-слов, пунктуации, лишних пробелов, чисел и возвращает лемматизированный текст.

```python
def lemmatize(doc):
    words = []
    for token in doc:
        if (token.is_stop != True) and (token.is_punct != True) and\
            (token.is_space != True) and (token.is_digit != True):
            words.append(token.lemma_)
    return ' '.join(words)
```

Вот некоторые соображения, которые могут помочь принять решение о выборе слоёв и их количества:

* Свёрточные слои `Conv1D`  эффективно работают с изображениями или текстовыми данными, где присутствует локальная структура текста (текст, где информация организуется внутри отдельных участков текста, таких как предложения и абзацы). Размер фильтров `kernel size` определяет размер области, которую рассматривает сверточный слой. Меньшие фильтры могут извлекать более мелкие особенности, в то время как большие фильтры могут улавливать более крупные особенности. Количество фильтров определяет количество различных признаков, которые извлекаются на каждом уровне свёртки.
* Слои пулинга `MaxPooling` или `AveragePooling` уменьшают размерность данных путём агрегации информации из предыдущего слоя. Они помогают уменьшить количество параметров модели и предотвратить переобучение.
* Полносвязанные слои `Dense` соединяются все нейроны из предыдущего слоя с каждым нейроном в текущем слое. Они обычно используются для обхединения информации из предыдущих слоёв перед выходом.

Обычно рекомендуется начать с относительно простой модели и постепенно усложнять её, добавляя больше слоёв или изменяя их конфигурацию, пока не будет достигнуто желаемое качество предсказаний.

Для задачи классификации текста с использованием CNN часто используется комбинация свёрточных слоёв для извлечения признаков из текста, за которым следуют слои пулинга для агрегации информации и полносвязные слои для классификации.

```python
from keras.model import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

# Подготовка данных
X = comment_data['text']
y = comment_data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Преобразование текста в последовательности индексов слов
X_train_sequences = w2v_model.wv[comment_data['text']]
X_test_sequences = w2v_model.wv[X_test]

# Инициализация весов word2vec
embedding_matrix = w2v_model.wv.vectors

num_words = len(w2v_model.wv.key_to_index) + 1
output_dim = w2v_model.vector_size

# Определение архитектуры модели
model = Sequential()
model.add(Embedding(input_dim = num_words, output_dim = embedding_dim, weights = [embedding_matrix], input_length = max_sequence_length, trainable = False))
model.add(Conv1D(filters = 128, kernel_size = 5, activation = 'relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(3, activation = 'softmax'))

# Компиляция модели
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Преобразование меток классов в категориальное представление
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

# Обучение модели
model.fit(X_train_sequences, y_train_categorical, epochs = 10, batch_size = 64, validation_split = 0.1)

#Оценка модели
loss, accuracy = model.evaluate(X_test_sequences, y_test_categorical)
print('Loss:', loss)
print('Accuracy:', accuracy)
```

Пример использования Optuna для оптимизации гиперпараметров модели

```python
import optuna
from tensorflow.keras.model import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1, Dense

def objective(trial):
    # Define the hyperparameters to optimize
    num_filters = trial.suggest_int('num_filters', 32, 128)
    kernel_size = trial.suggest_int('kernel_size', 3, 7)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'sigmoid'])

    model = Sequential([
        Conv1D(filters = num_filters, kernel_size = kernel_size, activation = activation, input_shape = input_shape),
        GlobalMaxPooling1D(),
        Dense(num_classes, activation = 'softmax')
    ])

    # Compile the model
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs = 10, batch_size = 32, validation_data = (X_val, y_val), verbose = 0)

    # Evaluate the model
    score = model.evaluate(X_val, y_val, verbose = 0)
    return score[1]

study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 100)

best_params = strudy.best_params
best_accuracy = study.best_value
```

`pad_sequences` позволяет привести данные к одной длине путём добавления нулевый значений в начало или конец последовательности
```python
padded_sequences = pad_sequences(sequences, maxlen = 100)
```

In [ ]:
comment_data['sentiment'].value_counts()

positive    203
negative    158
neutral      88
opponent     75
Name: sentiment, dtype: int64

In [ ]:
from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

from tensorflow import random
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

def lemmatize(text):
    words = []
    doc = nlp(text)
    for token in doc:
        if (token.is_punct != True) and (token.is_space != True) and (token.is_digit != True):
            words.append(token.lemma_)
    return ' '.join(words)

comment_data = pd.read_excel('values_df.xlsx')
comment_data['text'] = comment_data['value'].apply(lemmatize)

w2v_model = Word2Vec.load('word2vec-result-50')

# Подготовка данных
encoder = LabelEncoder()
comment_data['class'] = encoder.fit_transform(comment_data['sentiment'])

comment_data = shuffle(comment_data)

X = comment_data['text']
y = comment_data['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = comment_data['class'])

test_data = pd.DataFrame({'X_test': X_test, 'y_test': encoder.inverse_transform(y_test)})

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1

embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

max_length = 100
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')

# Инициализация весов word2vec
embedding_matrix = w2v_model.wv.vectors

num_words = len(w2v_model.wv.key_to_index) #embedding_matrix.shape[0]
output_dim = w2v_model.vector_size #embedding_matrix.shape[1]

In [ ]:
# Random Seed
random.set_seed(49)

# Определение архитектуры модели
model = Sequential()
model.add(Embedding(input_dim = num_words, output_dim = 100, weights = [embedding_matrix], input_length = 100, trainable = False))
model.add(Conv1D(filters = 98, kernel_size = 10, activation = 'sigmoid')),
model.add(Dropout(rate = 0.35)),
model.add(GlobalMaxPooling1D())
model.add(Dense(4, activation = 'softmax'))

# Компиляция модели
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

checkpoint_path = 'model_checkpoint_opponent.h5'
checkpoint = ModelCheckpoint(checkpoint_path, monitor = 'val_accuracy', save_best_only = True, mode = 'max', verbose = 1)

# Преобразование меток классов в категориальное представление
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

# Обучение модели
model.fit(X_train, y_train_categorical, epochs = 30, batch_size = 16, validation_data = (X_test, y_test_categorical), callbacks = [checkpoint])

In [ ]:
pd.DataFrame(best_params, index = [0]) #два свёрточных слоя 0.7809523940086365

,dropout_rate,random_seed,num_filters,kernel_size,activation,num_epochs,bath_size
0,0.395476,74,53,5,tanh,30,16


In [ ]:
pd.DataFrame(best_params, index = [0]) #один свёрточный слой 0.7904762029647827

,dropout_rate,random_seed,num_filters,kernel_size,activation,num_epochs,bath_size
0,0.353593,49,98,10,sigmoid,20,16


In [ ]:
model.load_weights(checkpoint_path)

loss, accuracy = model.evaluate(X_test, y_test_categorical)
print('Loss:', loss)
print('Accuracy:', accuracy) #  0.7809523940086365

4/4 [==============================] - 0s 23ms/step - loss: 0.7319 - accuracy: 0.7810
Loss: 0.7319156527519226
Accuracy: 0.7809523940086365


In [ ]:
%%time
def objective(trial):
    # Define the hyperparameters to optimize
    # dense_units = trial.suggest_int('dense_units', 32, 512)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    random_seed = trial.suggest_int('random_seed', 1, 100)
    num_filters = trial.suggest_int('num_filters', 32, 128)
    kernel_size = trial.suggest_int('kernel_size', 3, 10)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'sigmoid'])
    num_epochs = trial.suggest_categorical('num_epochs', [10, 20, 30])
    batch_size = trial.suggest_categorical('bath_size', [16, 32, 64])

    model = Sequential([
        Embedding(input_dim = num_words, output_dim = 100, weights = [embedding_matrix], input_length = 100, trainable = False),
        Conv1D(filters = num_filters, kernel_size = kernel_size, activation = activation),
        # Conv1D(filters = num_filters, kernel_size = kernel_size, activation = activation),
        # Conv1D(filters = num_filters, kernel_size = kernel_size, activation = activation),
        Dropout(rate = dropout_rate),
        GlobalMaxPooling1D(),
        # Dense(units = dense_units, activation = 'softmax'),
        Dense(4, activation = 'softmax')
    ])

    # Compile the model
    random.set_seed(random_seed)
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

    # Train the model
    model.fit(X_train, y_train_categorical, epochs = num_epochs, batch_size = batch_size, validation_data = (X_test, y_test_categorical), verbose = 0)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test_categorical, verbose = 1)
    return accuracy

study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 100)

In [ ]:
best_params = study.best_params
best_accuracy = study.best_value

print(best_accuracy)
print(best_params)

0.7904762029647827
{'dropout_rate': 0.35359335912346673, 'random_seed': 49, 'num_filters': 98, 'kernel_size': 10, 'activation': 'sigmoid', 'num_epochs': 20, 'bath_size': 16}


Если вы используете функцию `sparse_categorical_crossentropy` в TensorFlow для компиляции модели, то нет необходимости использовать `to_categorical` для преобразования меток классов в one-hot кодировку. Функция `sparse_categorical_crossentropy` автоматически принимает целочисленные метки классов и выполняет необходимую конвертацию внутри себя. </br>
Параметр `verbose` в функции `fit` библиотеки Keras определяет, какая информация будет выводиться в процессе обучения модели:
* `verbose=0`: В процессе обучения модели никакая информация не будет выводить на экран
* `verbose=1`: Выводится полоса прогресса progress bar в процессе обучения модели
* `verbose=2`: Выводится одна строка для каждой эпохи обучения, содержащая информациб о текущей эпохи и значениях выбранных метрик


In [ ]:
predictions = model.predict(X_test)

# Преобразуем предсказания в индексы классов, выбрав класс с наибольшей вероятностью
predicted_classes = np.argmax(predictions, axis = 1)

test_data['predicted_class'] = encoder.inverse_transform(predicted_classes)
test_data.sample(5)

3/3 [==============================] - 0s 10ms/step


,X_test,y_test,predicted_class
79,депозиты размещают,positive,neutral
280,по депозитам без изменений,neutral,positive
340,счет открыт + вкл выдана,positive,positive
266,продукты фт ),neutral,neutral
138,кредит 100 - 300 млн руб. + рамку на бг,neutral,neutral


In [ ]:
def form_sentiment_data(comment_data):
    sentiment_dict = {}
    # На вход подаём список из словарей
    for elem in comment_data:
        # Проходим по каждому из словарей списка и формируем sentiment_dict
        for key, value in elem.items():
            if key not in sentiment_dict:
                sentiment_dict[key] = 'neutral'
            # Переводим комментарий, значение словаря в уподобимый формат для модели
            tokenized_sentence = tokenizer.texts_to_sequences([lemmatize(value)])
            # Дополнений или обрезка последовательности
            padded_sentence = pad_sequences(tokenized_sentence, maxlen=100, padding='post')
            # Получаем предсказанный класс
            predicted_class = model.predict(padded_sentence)
            # Выбираем класс с наибольшей вероятностью
            predicted_class = np.argmax(predicted_class, axis = 1)
            # Переводим в текстовый формат
            predicted_class = encoder.inverse_transform(predicted_class)
            # Добавляем это значение по ключу Продукта
            sentiment_dict[key] = predicted_class
    return sentiment_dict

In [ ]:
create_partlist('кт. показатели снизились, за 2023 тоже, потребностей пока нет!')

[{'потребность': 'потребностей пока нет ! '}]

In [ ]:
def entity_extract(partlist):
    entlist = []
    for part in partlist:
        doc = nlp(part)
        for ent in doc.ents:
            if ent.label_ == 'PROD':
                entlist.append({ent.lemma_: part}) #ПРИВОДИМ К ФОРМАТУ СУЩНОСТЬ-ЗНАЧЕНИЕ
    return entlist

In [ ]:
pd.DataFrame(data_to_train['sentiment'].iloc[3])

,кредит,лизинг
0,positive,negative


In [ ]:
pd.DataFrame(data_to_train['sentiment'].iloc[9])

,кредитония,депозит,бг
0,positive,positive,negative
